In [31]:
# Install Prophet jika belum terpasang
# !pip install prophet

import pandas as pd
from prophet import Prophet
import matplotlib.pyplot as plt
from sklearn.metrics import mean_absolute_error, mean_squared_error
import numpy as np

# Load dataset
df = pd.read_csv("data/tiketcom_bestprice.csv", sep='|')

# Filter hanya rute dari Jakarta
df_jkt = df[df['origin'] == 'JKTC'].copy()
df_jkt['depart_date'] = pd.to_datetime(df_jkt['depart_date'])

# Ambil semua tujuan unik dari Jakarta
destinations = df_jkt['destination'].unique()

# Inisialisasi list untuk forecast dan evaluasi
all_forecasts = []
eval_results = []

# Loop untuk setiap tujuan
for dest in destinations:
    df_dest = df_jkt[df_jkt['destination'] == dest]
    
    # Group data harian dan ambil harga minimum per tanggal
    df_daily = df_dest.groupby('depart_date')['best_price'].min().reset_index()
    df_daily.columns = ['ds', 'y']
    
    if len(df_daily) < 30:
        continue  # skip jika data terlalu sedikit
    
    # Split train-test
    df_train = df_daily.iloc[:-7]
    df_test = df_daily.iloc[-7:]

    # Training Prophet
    model = Prophet(daily_seasonality=True)
    model.fit(df_train)
    
    # Forecast 7 hari ke depan
    future = model.make_future_dataframe(periods=7)
    forecast = model.predict(future)

    # Evaluasi
    forecast_test = forecast[['ds', 'yhat']].tail(7)
    y_true = df_test['y'].values
    y_pred = forecast_test['yhat'].values
    mae = mean_absolute_error(y_true, y_pred)
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    eval_results.append({'destination': dest, 'MAE': mae, 'RMSE': rmse})

    # Simpan prediksi
    forecast_7 = forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail(7)
    forecast_7['destination'] = dest
    all_forecasts.append(forecast_7)

# Gabungkan hasil
result_df = pd.concat(all_forecasts, ignore_index=True)
eval_df = pd.DataFrame(eval_results)

# Tentukan 10 tujuan terpopuler berdasarkan jumlah data (frekuensi) di df_jkt
top_dest = df_jkt['destination'].value_counts().head(10).index.tolist()

import plotly.graph_objs as go
from plotly.subplots import make_subplots

fig = make_subplots(rows=5, cols=2, subplot_titles=[f'Tujuan {d}' for d in top_dest])

for i, dest in enumerate(top_dest):
    subset = result_df[result_df['destination'] == dest]
    row = i // 2 + 1
    col = i % 2 + 1

    fig.add_trace(go.Scatter(x=subset['ds'], y=subset['yhat'], name=f'Forecast {dest}', mode='lines'), row=row, col=col)
    fig.add_trace(go.Scatter(x=subset['ds'], y=subset['yhat_upper'], name='Upper', mode='lines', line=dict(width=0), showlegend=False), row=row, col=col)
    fig.add_trace(go.Scatter(x=subset['ds'], y=subset['yhat_lower'], name='Lower', mode='lines', fill='tonexty', line=dict(width=0), fillcolor='rgba(173,216,230,0.3)', showlegend=False), row=row, col=col)

fig.update_layout(height=1200, width=1000, title_text="Forecast Harga Tiket 7 Hari ke 10 Tujuan Terpopuler dari Jakarta")
fig.show()

# Tampilkan tabel hasil forecasting untuk 10 tujuan terpopuler
for dest in top_dest:
    print(f"\n=== Forecast Harga Tiket 7 Hari ke Depan untuk Tujuan {dest} ===")
    display(result_df[result_df['destination'] == dest][['ds', 'yhat', 'yhat_lower', 'yhat_upper']].round(2))

import os
import pickle

# Buat folder untuk menyimpan model
os.makedirs("models", exist_ok=True)

# Loop untuk setiap tujuan
for dest in destinations:
    df_dest = df_jkt[df_jkt['destination'] == dest]
    
    df_daily = df_dest.groupby('depart_date')['best_price'].min().reset_index()
    df_daily.columns = ['ds', 'y']
    
    if len(df_daily) < 30:
        continue
    
    df_train = df_daily.iloc[:-7]
    df_test = df_daily.iloc[-7:]

    model = Prophet(daily_seasonality=True)
    model.fit(df_train)

    # 🚀 Export model Prophet ke file .pkl
    with open(f"models/prophet_model_{dest}.pkl", "wb") as f:
        pickle.dump(model, f)

    # Forecast
    future = model.make_future_dataframe(periods=7)
    forecast = model.predict(future)

    forecast_test = forecast[['ds', 'yhat']].tail(7)
    y_true = df_test['y'].values
    y_pred = forecast_test['yhat'].values
    mae = mean_absolute_error(y_true, y_pred)
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    eval_results.append({'destination': dest, 'MAE': mae, 'RMSE': rmse})

    forecast_7 = forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail(7)
    forecast_7['destination'] = dest
    all_forecasts.append(forecast_7)


# Simpan ke file jika perlu
# result_df.to_csv("forecast_7days_all_destinations.csv", index=False)
# eval_df.to_csv("evaluation_metrics_all_destinations.csv", index=False)


19:10:55 - cmdstanpy - INFO - Chain [1] start processing
19:10:55 - cmdstanpy - INFO - Chain [1] done processing
19:10:56 - cmdstanpy - INFO - Chain [1] start processing
19:10:56 - cmdstanpy - INFO - Chain [1] done processing
19:10:57 - cmdstanpy - INFO - Chain [1] start processing
19:10:57 - cmdstanpy - INFO - Chain [1] done processing
19:10:58 - cmdstanpy - INFO - Chain [1] start processing
19:10:58 - cmdstanpy - INFO - Chain [1] done processing
19:10:59 - cmdstanpy - INFO - Chain [1] start processing
19:10:59 - cmdstanpy - INFO - Chain [1] done processing
19:11:00 - cmdstanpy - INFO - Chain [1] start processing
19:11:00 - cmdstanpy - INFO - Chain [1] done processing
19:11:01 - cmdstanpy - INFO - Chain [1] start processing
19:11:01 - cmdstanpy - INFO - Chain [1] done processing
19:11:01 - cmdstanpy - INFO - Chain [1] start processing
19:11:02 - cmdstanpy - INFO - Chain [1] done processing
19:11:02 - cmdstanpy - INFO - Chain [1] start processing
19:11:02 - cmdstanpy - INFO - Chain [1]


=== Forecast Harga Tiket 7 Hari ke Depan untuk Tujuan JOG ===


,ds,yhat,yhat_lower,yhat_upper
35,2024-02-28,1179271.74,1118539.60,1238079.93
36,2024-02-29,1159670.18,1103113.89,1220507.75
37,2024-03-01,1168128.86,1108173.49,1228710.20
38,2024-03-02,1157739.38,1094830.40,1212627.42
39,2024-03-03,1173640.67,1114295.25,1233812.66
40,2024-03-04,1168617.95,1110432.00,1229040.25
41,2024-03-05,1157828.56,1097501.72,1215740.01



=== Forecast Harga Tiket 7 Hari ke Depan untuk Tujuan YIA ===


,ds,yhat,yhat_lower,yhat_upper
42,2024-03-14,351708.67,277639.74,425197.85
43,2024-03-15,390192.01,313971.72,462513.42
44,2024-03-16,346384.65,273017.69,423782.39
45,2024-03-17,353544.88,279397.89,431262.32
46,2024-03-18,346536.35,272033.99,418544.10
47,2024-03-19,346310.46,268516.78,419654.14
48,2024-03-20,348531.61,272582.40,419873.76


...


=== Forecast Harga Tiket 7 Hari ke Depan untuk Tujuan BDJ ===


,ds,yhat,yhat_lower,yhat_upper
133,2023-07-25,1087126.75,880068.86,1274123.32
134,2023-07-26,1117414.91,932186.71,1316072.95
135,2023-07-27,1113550.39,934996.03,1300570.96
136,2023-07-28,1104127.51,910145.67,1286472.27
137,2023-07-29,1111608.44,938283.13,1296759.72
138,2023-07-30,1121920.69,918683.97,1306655.39
139,2023-07-31,1116139.49,927657.25,1318199.63



=== Forecast Harga Tiket 7 Hari ke Depan untuk Tujuan MDC ===


,ds,yhat,yhat_lower,yhat_upper
161,2023-07-25,2910602.18,2576172.48,3237442.63
162,2023-07-26,2887547.84,2566446.52,3210359.15
163,2023-07-27,2977425.46,2640793.81,3326760.55
164,2023-07-28,2941618.42,2573631.83,3286581.57
165,2023-07-29,2909778.40,2576363.67,3250918.49
166,2023-07-30,2885128.83,2534028.25,3240218.86
167,2023-07-31,2879023.51,2514155.05,3205144.90


19:11:22 - cmdstanpy - INFO - Chain [1] start processing
19:11:23 - cmdstanpy - INFO - Chain [1] done processing
19:11:23 - cmdstanpy - INFO - Chain [1] start processing
19:11:23 - cmdstanpy - INFO - Chain [1] done processing
19:11:24 - cmdstanpy - INFO - Chain [1] start processing
19:11:24 - cmdstanpy - INFO - Chain [1] done processing
19:11:24 - cmdstanpy - INFO - Chain [1] start processing
19:11:25 - cmdstanpy - INFO - Chain [1] done processing
19:11:25 - cmdstanpy - INFO - Chain [1] start processing
19:11:26 - cmdstanpy - INFO - Chain [1] done processing
19:11:26 - cmdstanpy - INFO - Chain [1] start processing
19:11:27 - cmdstanpy - INFO - Chain [1] done processing
19:11:27 - cmdstanpy - INFO - Chain [1] start processing
19:11:27 - cmdstanpy - INFO - Chain [1] done processing
19:11:28 - cmdstanpy - INFO - Chain [1] start processing
19:11:28 - cmdstanpy - INFO - Chain [1] done processing
19:11:28 - cmdstanpy - INFO - Chain [1] start processing
19:11:29 - cmdstanpy - INFO - Chain [1]